In [1]:
! pip install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 949 kB/s eta 0:00:01


# Module

In [1]:
#Import Library
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

### Dataset

In [2]:
# MNIST 데이터셋 준비

mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()   

# 각 데이터 크기 출력
print(f"x_train.shape: {x_train.shape}")
print(f"y_train.shape: {y_train.shape}")
print(f"x_test.shape: {x_test.shape}")
print(f"y_test.shape: {y_test.shape}")

x_train.shape: (60000, 28, 28)
y_train.shape: (60000,)
x_test.shape: (10000, 28, 28)
y_test.shape: (10000,)


# Preprocessing

In [3]:
# Normalization - x
print('최소값:',np.min(x_train), ' 최대값:',np.max(x_train))

x_train_norm = x_train.astype('float32')/255.0
x_test_norm = x_test.astype('float32') / 255.0

print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))
#print(f"x_train.shape: {x_train.shape}")

최소값: 0  최대값: 255
최소값: 0.0  최대값: 1.0


In [4]:
# Normalization - y (One-hot-encoding)
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Model

In [5]:
from keras.layers import Input,Conv2D,Dropout, Dense,Flatten,GlobalAveragePooling2D,MaxPool2D
from keras.models import Model

#  tf.keras의 Sequential API를 이용하여 LeNet 모델 생성
model=keras.models.Sequential()
model.add(keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(32, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                25632     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                3

In [7]:
# 채널수 맞추기
print("Before Reshape - x_train_norm shape: {}".format(x_train_norm.shape))
print("Before Reshape - x_test_norm shape: {}".format(x_test_norm.shape))

x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 데이터갯수에 -1을 쓰면 reshape시 자동계산됩니다.
x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)

print("After Reshape - x_train_reshaped shape: {}".format(x_train_reshaped.shape))
print("After Reshape - x_test_reshaped shape: {}".format(x_test_reshaped.shape))


Before Reshape - x_train_norm shape: (60000, 28, 28)
Before Reshape - x_test_norm shape: (10000, 28, 28)
After Reshape - x_train_reshaped shape: (60000, 28, 28, 1)
After Reshape - x_test_reshaped shape: (10000, 28, 28, 1)


# Train Model

In [8]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])


In [9]:
%%time 

model.fit(x_train_reshaped, y_train, epochs=10)

Train on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 91us/sample - loss: 0.1816 - accuracy: 0.9452
Epoch 2/10
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0625 - accuracy: 0.9801
Epoch 3/10
60000/60000 [==============================] - 3s 44us/sample - loss: 0.0460 - accuracy: 0.9855
Epoch 4/10
60000/60000 [==============================] - 3s 45us/sample - loss: 0.0352 - accuracy: 0.9890
Epoch 5/10
60000/60000 [==============================] - 3s 48us/sample - loss: 0.0284 - accuracy: 0.9908
Epoch 6/10
60000/60000 [==============================] - 3s 49us/sample - loss: 0.0234 - accuracy: 0.9924
Epoch 7/10
60000/60000 [==============================] - 3s 47us/sample - loss: 0.0197 - accuracy: 0.9936
Epoch 8/10
60000/60000 [==============================] - 3s 53us/sample - loss: 0.0163 - accuracy: 0.9947
Epoch 9/10
60000/60000 [==============================] - 3s 47us/sample - loss: 0.0147 - accuracy: 0.9951
Epoch 10/10
60

# Model Evaluation

In [10]:
predictions = model.predict(x_test_reshaped)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on benign test example: {}%'.format(accuracy*100))

/home/aiffel/anaconda3/envs/aiffel2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Accuracy on benign test example: 98.92%


# Adversarial Examples

In [11]:
# ART
import art
from art.estimators.classification import KerasClassifier

#Wrap model with ART Keras Classifier
estimator = KerasClassifier(model=model, clip_values=(0,1), use_logits=True)

## 1-1) FGSM_origin
### 원본 이미지 x_test

In [12]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.1)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_reshaped,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.9722373485565186
CPU times: user 1.26 s, sys: 542 ms, total: 1.8 s
Wall time: 1.97 s


In [13]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 80.04%


## 1-2) FGSM_noise
### noise image -> x_test_noise

In [15]:
# noise 
noise = np.random.normal(0,0.05,x_test.shape)
x_test_noise1 = x_test_norm + noise
x_test_noise1_reshape = x_test_noise1.reshape(-1,28,28,1)

In [16]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise1_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.0409772396087646
CPU times: user 1.26 s, sys: 119 ms, total: 1.38 s
Wall time: 1.04 s


In [17]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 29.65%


In [20]:
# noise 
noise = np.random.normal(0,0.05,x_test.shape)
x_test_noise = x_test_norm + noise
x_test_noise_reshape = x_test_noise.reshape(-1,28,28,1)

In [21]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1782159805297852
CPU times: user 1.44 s, sys: 138 ms, total: 1.58 s
Wall time: 1.18 s


In [22]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 29.970000000000002%


In [23]:
# noise 
noise = np.random.normal(0,0.05,x_test.shape)
x_test_noise = x_test_norm + noise
x_test_noise_reshape = x_test_noise.reshape(-1,28,28,1)

In [24]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1004486083984375
CPU times: user 1.33 s, sys: 121 ms, total: 1.45 s
Wall time: 1.1 s


In [25]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 30.09%


In [26]:
# noise 
noise = np.random.normal(0,0.05,x_test.shape)
x_test_noise = x_test_norm + noise
x_test_noise_reshape = x_test_noise.reshape(-1,28,28,1)

In [27]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1194097995758057
CPU times: user 1.41 s, sys: 134 ms, total: 1.54 s
Wall time: 1.12 s


In [28]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 30.509999999999998%


In [29]:
# noise 
noise = np.random.normal(0,0.05,x_test.shape)
x_test_noise = x_test_norm + noise
x_test_noise_reshape = x_test_noise.reshape(-1,28,28,1)

In [30]:
%%time 

start = time.time() # 시작 시간 저장

from art.attacks.evasion import FastGradientMethod
#Create FGSM Instance
attack_fgsm = FastGradientMethod(estimator, eps=0.2)

#Create Adversarial Examples
examples_fgsm = attack_fgsm.generate(x_test_noise_reshape,y_test)

print("time:",time.time()-start) # 실행시간

time: 1.1018917560577393
CPU times: user 1.33 s, sys: 113 ms, total: 1.45 s
Wall time: 1.1 s


In [31]:
prediction_fgsm = model.predict(examples_fgsm)

acc_fgsm = np.sum(np.argmax(prediction_fgsm, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print('Accuracy on Adversarial Exmaples: {}%'.format(acc_fgsm * 100))

Accuracy on Adversarial Exmaples: 29.959999999999997%
